In [ ]:
!pip install ortools

In [12]:
from ortools.linear_solver import pywraplp
from numpy import average, std
import json
import pandas as pd
import requests

In [13]:
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)

In [4]:
with open("./functions/neighbors_with_9_minute_station.json", "r") as f:
    neighbors_with_9_minute_station = json.load(f)

In [17]:
neighbourhoods = list(neighbors_with_9_minute_station.keys())
K = len(neighbourhoods)
shifts = ["9 AM to 6 PM", "6 PM until 9 AM"]
I = len(shifts)
stations = list(distances_stations_ntas_db.keys())
L = len(stations)
vehicles = ["engine", "squad", "ladder", "hazardous", "rescue"]
J = len(vehicles)

In [10]:
estimated_demand = pd.read_csv("./functions/estimated_demand.csv")
estimated_demand

,Unnamed: 0,nta,engines,squads,ladders,ha,rescue,shift
0,0,BK17,4.934514,0.188820,3.806345,0.028029,0.113781,1
1,1,BK25,3.671613,0.159006,2.610501,0.018974,0.099957,1
2,2,BK27,2.396166,0.125880,1.681814,0.011643,0.062739,1
3,3,BK28,5.043240,0.208696,3.684216,0.028029,0.129732,1
4,4,BK30,3.378887,0.162319,2.236482,0.016818,0.086134,1
...,...,...,...,...,...,...,...,...
84,84,QN52,1.802352,0.023188,1.094070,0.009487,0.005317,0
85,85,QN55,4.871787,0.039752,2.948900,0.025011,0.057422,0
86,86,QN62,1.250356,0.013251,0.844724,0.009918,0.011697,0
87,87,SI07,4.290518,0.046377,2.554526,0.038379,0.034028,0


In [21]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)
capacity_at_station = {}
for st in stations_db:
    capacity_at_station[stations.index(st["address"])] = st["capacity"]

In [18]:
def index2shift(i):
    return shifts[i]

def index2vehicle(j):
    return vehicles[j]

def index2neighbourhoods(k):
    return neighbourhoods[k]

def index2station(l):
    return stations[l]

In [ ]:
solver_scip = pywraplp.Solver.CreateSolver("SCIP") # Example, creating a SCIP solver
solver_cbc = pywraplp.Solver.CreateSolver("CBC") # Example, creating a CBC solver
solver_glop = pywraplp.Solver.CreateSolver("GLOP") # Example, creating a SCIP solver

In [ ]:
solver = solver_glop

variables = {}

for i in range(I):
  for j in range(J):
    for k in range(K):
      for l in range(L):
        v = solver.NumVar(0, solver.infinity(), "Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " to cover the neighbourhood " + index2neighbourhood(k) + " in shift " + index2shift(i))
        variables[(i,j,k,l)] = v

In [ ]:
objective = solver.Objective()
objective.SetMinimization()
for i in range(number_origins):
  for j in range(number_destinations):
    v = variables[(i,j)]
    cost = transportation_cost_per_unit[(i,j)]
    objective.SetCoefficient(v, cost)

In [ ]:
station_capacity = []

for l in range(L):
  capacity = capacity_at_station[l]
  for i in range(I):
    c = solver.Constraint(-solver.infinity(), capacity)
    for j in range(J):
      for k in range(K):
        v = X[(i,j,k,l)]
        c.SetCoefficient(v, 1)
    station_capacity.append(c)

In [ ]:
nine_minutes = []

for k in range(K):
  for i in range(I):
    c = solver.Constraint(1, solver.infinity())
    for l in range(L):
      is_inside = station_inside[(k,l)]
      for j in range(J):
        v = X[(i,j,k,l)]
        c.SetCoefficient(v, is_inside)
    nine_minutes.append(c)

In [ ]:
solver = pywraplp.Solver.CreateSolver("GLOP")

variables = {}

for i in range(number_origins):
  for j in range(number_destinations):
    v = solver.NumVar(0, solver.infinity(), "Units transported between: " + str((i,j)))
    variables[(i,j)] = v

objective = solver.Objective()
objective.SetMinimization()
for i in range(number_origins):
  for j in range(number_destinations):
    v = variables[(i,j)]
    cost = transportation_cost_per_unit[(i,j)]
    objective.SetCoefficient(v, cost)

stock_constraints = []

for i in range(number_origins):
  stock = stock_at_origin[i]
  c = solver.Constraint(-solver.infinity(), stock)
  for j in range(number_destinations):
    v = variables[(i,j)]
    c.SetCoefficient(v, 1)
  stock_constraints.append(c)

demand_constraints = []

for j in range(number_destinations):
  min_demand = demand_at_destination[j]
  c = solver.Constraint(min_demand, solver.infinity())
  for i in range(number_origins):
    v = variables[(i,j)]
    c.SetCoefficient(v,1)
  demand_constraints.append(c)

result = solver.Solve()

if result == solver.ABNORMAL :
  print("Execution finished by an error")
elif result == solver.FEASIBLE :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(number_origins):
    for j in range(number_destinations):
      v = variables[(i,j)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE :
  print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED :
  print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(number_origins):
    for j in range(number_destinations):
      v = variables[(i,j)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED :
  print("The solution is unbounded")
else :
  print("Unknown error code")